In [41]:
# libraries
#conda install -c anaconda basemap

import pandas as pd
import numpy as np
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup as beauty

import warnings
warnings.filterwarnings('ignore')

In [42]:
cities = pd.read_csv('newRes90_clean.csv', index_col = 0)
coord = pd.read_csv('coordiantes.csv', index_col = 0)
coord['city'] = coord.index
cities.shape, coord.shape
cities = cities.join(coord)

In [43]:
cities['tier'].unique()

array(['Tier 4', 'Tier 3', 'Tier 1', 'Tier 2'], dtype=object)

In [44]:
import plotly.express as px

colour_map = {'Tier 1': 'white', 'Tier 2': 'aquamarine', 'Tier 3': 'azure', 'Tier 4': 'blue'}
#colour_map = {'Tier 1': 'Tier 1', 'Tier 2': 'Tier 2', 'Tier 3': 'Tier 3', 'Tier 4': 'Tier 4'}


fig = px.scatter_mapbox(cities, lat="Lon", lon="Lat", hover_name= 'chinese_name', hover_data=['city'],
                         zoom=2.8, height=500, \
                        #color=cities['tier'].map(colour_map) ,\
                        color_discrete_sequence=cities['tier'].map(colour_map),\
                       size=(cities.loc[:,'202004']- 5))
fig.update_layout(mapbox_style="carto-darkmatter") #stamen-terrain, open-street-map,carto-positron,carto-darkmatter
'''fig.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])'''
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [45]:
from ipywidgets import interactive
import matplotlib.font_manager as mfm
font_path = "/usr/share/fonts/truetype/NotoSansSC-Regular.otf"
font_path = "/usr/share/fonts/truetype/DroidSansFallbackFull.ttf"
font_path = "/usr/share/fonts/truetype/NotoSansTC-Bold.otf"
prop = mfm.FontProperties(fname=font_path)
#plt.rcParams['font.family']= 'DejaVu Sans'

def price():
   
    def _cdef(Date,City, Price):
        c = City       
        p = Price
        t = Date
        
        
        fig = plt.figure(figsize = (15,4))
        ax = fig.add_subplot(1,1,1)
        for i, l  in zip([new,sec],['new', '2nd hand']):
            i = i.loc[c, t:]
            plt.plot(i.index, i.values, label= c+' '+l)
            plt.fill_between(i.index, i.values,
                100, # the price of reference, buying price eg. exact No. also good
                where = (i.values > 100), #shows only when higher then the buying price
                facecolor = 'g',
                alpha = 0.5
                )
            plt.fill_between(i.index, i.values,
                100, # the price of reference, buying price eg. exact No. also good
                where = (i.values <= 100), #shows only when higher then the buying price
                facecolor = 'r',
                alpha = 0.5
                )

        plt.xticks(rotation = 45)
        plt.legend(loc = (0.8,0.85))
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.axhline(100,color = 'red', linewidth = 1)
        plt.suptitle('Property price index change between {} and today'.format(t), fontsize = 16)
        plt.title('{}至今天之间的房地产价格变化'.format(t), fontproperties=prop, fontsize = 14)
        
        
        plt.show()
        
        #price calculation
        p1 = p
        if d == 'new':
            plist = new.loc[c,t:]
        else:
            plist = sec.loc[c,t:]
        for i in plist:
            p1 *=(float(i)/100)
        pp1 = p1/p*100
        
        
        print('The price increase in the last {} month is {:.2f} %, value {:0.0f}'.format(len(plist), pp1, p1))
        print('最近{}个月的价格上涨值为{:.2f} %, RMB {:0.0f}'.format(len(plist), pp1, p1))
        
        return
    
    new = pd.read_csv('newRes90_clean.csv', index_col = 0)
    sec = pd.read_csv('secRes90_clean.csv', index_col = 0)

    #t = input('date 201810: ')
    d = input('Property type: ')
    c = interactive(_cdef,Date = sec.columns, City = sec.index, Price = (200000,1500000,50000))
    #display(c)
    return c

In [46]:
price()

Property type: 


interactive(children=(Dropdown(description='Date', options=('201407', '201408', '201409', '201410', '201411', …

In [70]:
inflation = [
            { 'year':'2019' ,'inflation':2.7},
            { 'year':'2018' ,'inflation': 2.1},
            { 'year':'2017' ,'inflation':1.56},
            { 'year':'2016' ,'inflation': 2},
            { 'year':'2015' ,'inflation':1.44},
            { 'year':'2014' ,'inflation':1.99}
            ]

In [53]:
def get_y(js):
    return js['inflation']

m = sorted(inflation, key= get_y)
m = min(inflation, key= lambda x: x['inflation'])
m

{'year': '2015', 'inflation': 1.44}

In [54]:
for i in m:
    print(i)

year
inflation


In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import random as rd
import numpy as np

In [47]:
from ipywidgets import interactive
import matplotlib.font_manager as mfm
font_path = "/usr/share/fonts/truetype/NotoSansSC-Regular.otf"
font_path = "/usr/share/fonts/truetype/DroidSansFallbackFull.ttf"
font_path = "/usr/share/fonts/truetype/NotoSansTC-Bold.otf"
prop = mfm.FontProperties(fname=font_path)
#plt.rcParams['font.family']= 'DejaVu Sans'

#new = pd.read_csv('newRes90_clean.csv', index_col = 0)
sec = pd.read_csv('secRes90_clean.csv', index_col = 0)

with open('inflation.json') as f:
    inf = json.load(f)
inflist = {i['year']:i['inflation'] for i in inf}
rate = pd.DataFrame(pd.to_datetime(sec.columns, format='%Y%m').year)
rate.columns = ['year']
rate['inf'] = rate['year'].map(inflist)

def price():
   
    def _cdef(Date,City, Price):
        c = City       
        p = Price
        t = Date
        
        def _colour(i):
            if i > 100:
                 return 'green'
            else: return 'red'
        
        fig = plt.figure(figsize = (15,4))
        ax = fig.add_subplot(1,1,1)
        for i, l  in zip([sec],['2nd hand']):
            i = i.loc[c, t:]
            plt.bar(i.index, i.values-100, label= c+' '+l, color = i.map(_colour), bottom = 100)
            

        plt.xticks(rotation = 45)
        plt.legend(loc = (0.8,0.85))
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.axhline(100,color = 'red', linewidth = 1)
        plt.suptitle('Property price index change between {} and today'.format(t), fontsize = 16)
        plt.title('{}至今天之间的房地产价格变化'.format(t), fontproperties=prop, fontsize = 14)
        
        
        plt.show()
        
        #price calculation
        p1 = p
        r1 = p
        if d == 'new':
            plist = new.loc[c,t:]
        else:
            plist = sec.loc[c,t:]
            rlist = rate['inf'][len(plist)-1:]
        for i,r in zip(plist,rlist):
            p1 *=(float(i)/100)
            r1 *=((float(i)/100)*(1-r100))
            print(r1,i,r)
        pp1 = p1/p*100
        rr1 = r1/p*100
        
        
        
        print('The price increase in the last {} month is {:.2f} %, value {:0.0f}'.format(len(plist), pp1, p1))
        print('Inflation corrected increase {}%, value: {:0.0f}'.format(rr1,r1) )
        print('最近{}个月的价格上涨值为{:.2f} %, RMB {:0.0f}'.format(len(plist), pp1, p1))
        
        return
    
    

    #t = input('date 201810: ')
    d = input('Property type: ')
    c = interactive(_cdef,Date = sec.columns, City = sec.index, Price = (200000,1500000,50000))
    #display(c)
    return c

In [48]:
price()

Property type: 


interactive(children=(Dropdown(description='Date', options=('201407', '201408', '201409', '201410', '201411', …

In [33]:
new = pd.read_csv('newRes90_clean.csv', index_col = 0)
new.shape

(70, 70)

In [34]:
new.dtypes.to_frame().loc[new.dtypes != 'float64']

Empty DataFrame
Columns: [0]
Index: []

In [35]:
new.loc['Anqing','201903']

nan

In [59]:
inf = pd.DataFrame(pd.to_datetime(new.columns, format ='%Y%m').year)
inf.columns = ['year']

In [93]:
inf['inflation'] = inf['year'].map(rate)

In [88]:
rate = {int(i['year']):i['inflation']/12 for i in inflation}

In [92]:
rate[2020] = 3.6/12

In [94]:
inf

,year,inflation
0,2014,0.165833
1,2014,0.165833
2,2014,0.165833
3,2014,0.165833
4,2014,0.165833
...,...,...
65,2020,0.300000
66,2020,0.300000
67,2020,0.300000
68,2020,0.300000


In [141]:
inf

[{'year': 2019, 'inflation': 2.7},
 {'year': 2018, 'inflation': 2.1},
 {'year': 2017, 'inflation': 1.56},
 {'year': 2016, 'inflation': 2},
 {'year': 2015, 'inflation': 1.44},
 {'year': 2014, 'inflation': 1.99},
 {'year': 2020, 'inflation': 3.6, 'remark': 'estimated'}]

In [33]:
import numpy as np

newlist = []
for n in l:
    i = 1
    while i< 13:
        n = np.sqrt(n)
        i += 1
    newlist.append(n)

1 1.6431676725154984
2 1.2818610191887023
3 1.1321930132219957
4 1.0640455879434845
5 1.0315258542292987
6 1.0156406127313433
7 1.0077899645915032
8 1.0038874262543103
9 1.0019418277795924
10 1.0009704430099784
11 1.0004851038421203
12 1.0002425225124756
1 1.7320508075688772
2 1.3160740129524924
3 1.147202690439877
4 1.0710754830729146
5 1.0349277670798647
6 1.0173139963058921
7 1.0086198472694716
8 1.0043006757288733
9 1.0021480308461785
10 1.0010734392871377
11 1.000536575686835
12 1.0002682518638863


[1.0002425225124756, 1.0002682518638863]

In [40]:
i = 1
v = float(100)
while i < 12:
    v = v*(1 - (n-1))
    print(i,v)
    i += 1

1 99.97317481361137
2 99.94635682312898
3 99.91954602662253
4 99.8927424221622
5 99.86594600781874
6 99.83915678166336
7 99.81237474176784
8 99.78559988620444
9 99.75883221304595
10 99.73207172036567
11 99.70531840623744


In [37]:
n

-0.0002682518638863485